# Aula: O "E se forem Palavras?" (Deep Learning para NLP)

**A Ponte:** Já sabemos classificar imagens. Agora, vamos classificar texto. O desafio é transformar palavras, que são categóricas, em números que capturem seu significado.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

ImportError: cannot import name 'runtime_version' from 'google.protobuf' (/Users/leonardoraiz/Documents/Leonardo/FATEC/Aulas/IA/Outros/Aula 04 - Sistemas Especialistas /.venv/lib/python3.9/site-packages/google/protobuf/__init__.py)

### Passo 1: Carregar os Dados (IMDb)

O Keras já tem o dataset IMDb, que contém 50.000 críticas de filmes, já pré-processadas onde cada palavra é um número (índice).

In [ ]:
VOCAB_SIZE = 10000 # Pegar as 10.000 palavras mais comuns

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=VOCAB_SIZE)

print(f"Temos {len(x_train)} críticas de treino.")
print("\nExemplo de uma crítica (como números):")
print(x_train[0])

### Passo 2: Pré-Processamento (Padding)

As críticas têm tamanhos diferentes, mas as redes neurais precisam de entradas de tamanho fixo. Vamos "padronizar" todas as críticas para ter, no máximo, 250 palavras. Se for menor, preenchemos com zeros (`padding`).

In [ ]:
MAX_LEN = 250

x_train_pad = pad_sequences(x_train, maxlen=MAX_LEN, padding='post')
x_test_pad = pad_sequences(x_test, maxlen=MAX_LEN, padding='post')

print("\nCrítica depois do 'padding' para 250 palavras:")
print(x_train_pad[0])

### Passo 3: Construir o Modelo (RNN com LSTM)

Esta arquitetura é o padrão para problemas de sequência:
1.  **`Embedding`**: A camada que transforma os índices (números) em vetores densos (o "significado").
2.  **`LSTM`**: A camada de "memória" que lê a sequência de vetores de palavras.
3.  **`Dense`**: A camada final de classificação (Positivo/Negativo).

In [ ]:
EMBEDDING_DIM = 16 # Dimensão do vetor de cada palavra

model = keras.Sequential([
    # 1. Camada de Embedding: Transforma os 10.000 índices em vetores de 16 dimensões
    layers.Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_LEN),
    
    # 2. Camada LSTM: A rede com "memória" para ler a sequência
    layers.LSTM(64),
    
    # 3. Camada Densa de classificação
    layers.Dense(64, activation='relu'),
    
    # 4. Camada de Saída (1 neurônio - Positivo ou Negativo)
    layers.Dense(1, activation='sigmoid')
])

model.summary()

# Compilar o modelo
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])


### Passo 4: Treinar e Avaliar

In [ ]:
print("Iniciando o treinamento do modelo de NLP...")

history = model.fit(x_train_pad, y_train, 
                    epochs=5, 
                    batch_size=128,
                    validation_data=(x_test_pad, y_test))

print("Treinamento Concluído!")

# Avaliar
test_loss, test_accuracy = model.evaluate(x_test_pad, y_test)
print(f"\nAcurácia no teste (críticas inéditas): {test_accuracy * 100:.2f}%")